<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220518_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
import os
import tensorflow_datasets as tfds

# TPU 초기화
- TPU는 사용자의 파이썬 프로그램을 실행하는 로컬 프로세스와 다른 Cloud TPU 작업자이다.
- 즉 원격 클러스터에 연결하고 TPU를 초기화하려면 일부 초기화 작업을 수행해야 한다.
- `tf.distribute.cluster_resolver.TPUClusterResolver`에 대한 `tpu` 인수는 전용 특수 주소다. 
  - 코랩과 같이 `Google Compute Engine(GCE)`에서 실행한다면 Cloud TPU의 이름을 전달한다.
  - TPU 초기화 코드는 프로그램 시작 부분에 있어야 한다.

* 코랩에서 TPU를 할당하지 못하는 경우가 있는데, 여러 번 반복하면 연결됨

In [4]:
# TPU 초기화 작업 : 이것만 따로 저장해놓고 써도 될듯
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='') # 코랩에서 그냥 이대로 실행됩니다
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices : ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.2.81.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.2.81.138:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices :  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


## 수동 장치 배치
- TPU 초기화 후 수동 장치 배치를 사용, 단일 TPU 장치에 계산을 배치할 수 있다.

In [5]:
# 사용 방식은 동일함
a = tf.constant([[1., 2., 3.], [4., 5., 6.]])
b = tf.constant([[1., 2.], [3., 4.], [5., 6.]])

with tf.device('/TPU:0'):
  c = tf.matmul(a, b)

print("c device : ", c.device)
print(c)

c device :  /job:worker/replica:0/task:0/device:TPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


## 유통 전략 : `tf.distribute.TPUStrategy`
- 데이터 병렬 방식으로 여러 TPU에서 모델이 실행됨
- 이를 위해 쓰이는 게 전략임

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [7]:
# 모든 TPU 코어에서 실행시키려면 strategy.run API에 전달함
@tf.function
def matmul_fn(x, y):
  z = tf.matmul(x, y)
  return z

z = strategy.run(matmul_fn, args=(a,b))
print(z)

PerReplica:{
  0: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  1: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  2: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  3: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  4: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  5: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  6: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  7: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
}


# TPU를 이용한 분류(예제)
- `tf.distribute.TPUStrategy`를 사용해 Cloud TPU에서 케라스 모델을 학습시킬 수 있다.

In [8]:
# 케라스 모델 정의
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation = 'relu', input_shape = (28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation = 'relu',),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation = 'relu'),
       tf.keras.layers.Dense(128, activation = 'relu'),
       tf.keras.layers.Dense(10)
      ])

### 데이터셋 로드
`tf.data.Dataset` API의 효율적인 사용은 매우 중요함
- 대부분의 실험에서 데이터세트에서 읽은 모든 데이터 파일을 GCS 버킷에 저장해야 함
  - 그리고 데이터를 `TFRecord` 형식으로 변환하고, `tf.data.TFRecordDataset`을 사용하여 읽는 방법이 좋다. `TFRecord, tf.Example 자습서`가 따로 있음.
-`tf.data.Dataset.cache`로 작은 데이터 세트를 메모리에 로드하는 것도 가능함.

In [9]:
# 데이터세트 로드
def get_dataset(batch_size, is_training = True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name = 'mnist', split = split, with_info = True,
                            as_supervised = True, try_gcs = True)
  
  # Normalize
  def scale(image, label):
    image = tf.casts(image, tf.float32)
    image /= 255.0
    return image, label

  # 무한한 데이터셋의 장점 : 마지막 에포크에 들어가는 데이터의 수를 걱정할 필요가 없다
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()

  dataset = dataset.batch(batch_size)

  return dataset

## 학습

In [10]:
# TPU 관련 코드가 없죠
with strategy.scope():
  model = create_model()
  model.compile(optimizer = 'adam',
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['sparse_categorical_accuracy'])
  
batch_size = 200
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

train_dataset = get_dataset(batch_size, is_training = True)
test_dataset = get_dataset(batch_size, is_training = False)

model.fit(train_dataset,
          epochs = 5,
          steps_per_epoch = steps_per_epoch,
          validation_data = test_dataset,
          validation_steps = validation_steps)

Epoch 1/5
300/300 [==============================] - 23s 46ms/step - loss: 1.2020 - sparse_categorical_accuracy: 0.9232 - val_loss: 0.0820 - val_sparse_categorical_accuracy: 0.9753
Epoch 2/5
300/300 [==============================] - 11s 35ms/step - loss: 0.0469 - sparse_categorical_accuracy: 0.9853 - val_loss: 0.0764 - val_sparse_categorical_accuracy: 0.9790
Epoch 3/5
300/300 [==============================] - 12s 40ms/step - loss: 0.0294 - sparse_categorical_accuracy: 0.9904 - val_loss: 0.0759 - val_sparse_categorical_accuracy: 0.9798
Epoch 4/5
300/300 [==============================] - 12s 39ms/step - loss: 0.0215 - sparse_categorical_accuracy: 0.9929 - val_loss: 0.0890 - val_sparse_categorical_accuracy: 0.9798
Epoch 5/5
300/300 [==============================] - 12s 39ms/step - loss: 0.0188 - sparse_categorical_accuracy: 0.9939 - val_loss: 0.1153 - val_sparse_categorical_accuracy: 0.9759


### TPU 성능 최대화하기
- 인수 `steps_per_execution`을 `Model.compile`에 전달한다. 

In [12]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer = 'adam',
                steps_per_execution = 50, # 값은 2 ~ steps_per_epoch 사이로 전달한다.
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['sparse_categorical_accuracy'])
  
model.fit(train_dataset, epochs = 5,
          steps_per_epoch = steps_per_epoch,
          validation_data = test_dataset,
          validation_steps = validation_steps)

Epoch 1/5
300/300 [==============================] - 17s 55ms/step - loss: 2.4735 - sparse_categorical_accuracy: 0.9262 - val_loss: 0.0824 - val_sparse_categorical_accuracy: 0.9729
Epoch 2/5
300/300 [==============================] - 4s 14ms/step - loss: 0.0554 - sparse_categorical_accuracy: 0.9824 - val_loss: 0.0728 - val_sparse_categorical_accuracy: 0.9772
Epoch 3/5
300/300 [==============================] - 4s 14ms/step - loss: 0.0320 - sparse_categorical_accuracy: 0.9900 - val_loss: 0.0857 - val_sparse_categorical_accuracy: 0.9771
Epoch 4/5
300/300 [==============================] - 4s 15ms/step - loss: 0.0233 - sparse_categorical_accuracy: 0.9919 - val_loss: 0.0888 - val_sparse_categorical_accuracy: 0.9803
Epoch 5/5
300/300 [==============================] - 4s 14ms/step - loss: 0.0235 - sparse_categorical_accuracy: 0.9931 - val_loss: 0.1093 - val_sparse_categorical_accuracy: 0.9762


- 예제에서는 70초 vs 33초의 성능 차이가 났음

## 사용자 지정 훈련 루프로 훈련

In [22]:
with strategy.scope():
  model = create_model()
  optimizer = tf.keras.optimizers.Adam()
  training_loss = tf.keras.metrics.Mean('training_loss', dtype = tf.float32)
  training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      'training_accuracy', dtype = tf.float32
  )

# 전달된 배치 크기 : 복제본당 배치 크기임(전역 배치 크기 아님)
per_replica_batch_size = batch_size // strategy.num_replicas_in_sync # 각 TPU에 할당하는 batch 사이즈 계싼 & 분배

# 함수가 지정된 데이터 세트를 배포
train_dataset = strategy.experimental_distribute_datasets_from_function(
    lambda _: get_dataset(per_replica_batch_size, is_training = True)
)

@tf.function
def train_step(iterator):
  def step_fn(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape:
      logits = model(images, training = True)
      loss = tf.keras.losses.sparse_categorical_crossentropy(
          labels, logits, from_logits = True
      )
      loss = tf.nn.compute_average_loss(loss, global_batch_size = batch_size)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)

  strategy.run(step_fn, args = (next(iterator), ))

In [24]:
# 훈련 루프 실행
steps_per_eval = 10000 // batch_size

train_iterator = iter(train_dataset)
for epoch in range(5):
  print('Epoch : {} / 5'.format(epoch))

  for step in range(steps_per_epoch):
    train_step(train_iterator)

  print("Current step : {}, training loss : {}, accuracy : {}%".format(optimizer.iterations.numpy(),
                                                                       round(float(training_loss.result()), 4),
                                                                       round(float(training_accuracy.result()) * 100, 2)))
  training_loss.reset_states()
  training_accuracy.reset_states()

Epoch : 0 / 5
Current step : 600, training loss : 1.4805, accuracy : 92.54%
Epoch : 1 / 5
Current step : 900, training loss : 0.0369, accuracy : 98.85%
Epoch : 2 / 5
Current step : 1200, training loss : 0.025, accuracy : 99.15%
Epoch : 3 / 5
Current step : 1500, training loss : 0.0188, accuracy : 99.34%
Epoch : 4 / 5
Current step : 1800, training loss : 0.0162, accuracy : 99.48%


### `tf.function` 내부 여러 단계로 성능 향상
- `tf.range` 내부 `tf.function`으로 `strategy.run` 호출을 래핑, AutoGraph는 이를 TPU 작업자의 `tf.while_loop`으로 변환한다.

In [32]:
@tf.function
def train_multiple_steps(iterator, steps):

  def step_fn(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape:
      logits = model(images, training = True)
      loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)
      loss = tf.nn.compute_average_loss(loss, global_batch_size = batch_size)
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)

  # tf.range 내부의 tf.function으로 strategy.run 호출을 래핑
  # AutoGraph는 이를 TPU 작업자의 tf.while_loop으로 변환한다.
  for _ in tf.range(steps):
    strategy.run(step_fn, args = (next(iterator), ))    
    
train_multiple_steps(train_iterator, tf.convert_to_tensor(steps_per_epoch))


print("Current step : {}, Training Loss : {}, Accuracy : {} %".format(
    optimizer.iterations.numpy(),
    round(float(training_loss.result()), 4),
    round(float(training_accuracy.result()) * 100, 2))
)

Current step : 2100, Training Loss : 0.0138, Accuracy : 99.54 %
